In [25]:
import geopandas as gp
import pandas as pd
import os
import maup

# 2020 Census Demographics Aggregated to Louisiana 2023 Precincts

## Approach:
- Load in 2023 LA precinct file
- Maup census blocks to 2023 LA precinct files
- Export the file as a CSV and SHP file.

## Links to datasets used:
- [2020 Louisiana Census Block Data](https://redistrictingdatahub.org/dataset/louisiana-block-pl-94171-2020/)
- [2023 Louisiana Precinct Boundaries and Election Results](https://redistrictingdatahub.org/dataset/louisiana-2023-primary-and-general-election-precinct-level-results-and-boundaries/)

In [26]:
# Load in raw data
la_2023_precs = gp.read_file("./raw-from-source/la_2023_gen_prim_pber/la_2023_gen_prim_pber.shp")
la_blocks = gp.read_file("./raw-from-source/la_pl2020_b/la_pl2020_b.shp")

# Change index to make block assignment to precinct easier
la_2023_precs.index = la_2023_precs["UNIQUE_ID"]
la_blocks["PREC"] = maup.assign(la_blocks, la_2023_precs)

# Export block assignments to csv
la_blocks[["GEOID20","PREC"]].to_csv("./la_2023_gen_prim_pber_demos/BAF.csv", index = False)

In [9]:
# Group blocks by precinct and clean indices for join
prec_demos = la_blocks.groupby("PREC").sum()
prec_demos.reset_index(inplace = True, drop = False)

la_2023_precs.reset_index(inplace = True, drop = True)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_67955/2824390904.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  prec_demos = la_blocks.groupby("PREC").sum()


In [33]:
# Join together and drop "_merge" check column
comb = gp.GeoDataFrame(pd.merge(la_2023_precs, prec_demos, how = "outer", left_on = "UNIQUE_ID", right_on = "PREC", indicator = True))
comb.drop(["_merge"], axis = 1, inplace = True)

# Export as .shp
comb.to_file("./la_2023_gen_prim_pber_demos/la_2023_gen_prim_pber_demos.shp")

ValueError: 'UNIQUE_ID' is both an index level and a column label, which is ambiguous.